# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [50]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [51]:
device = '/device:gpu:0'

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [52]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [53]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [54]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [55]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [56]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Задаем инициализатор весов
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Создаем слой для выравнивания данных
        self.flatten = tf.keras.layers.Flatten()

        # Создаем первый сверточный слой с активационной функцией ReLU
        self.conv1 = tf.keras.layers.Conv2D(channel_1, [5, 5], [1, 1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')

        # Создаем второй сверточный слой с активационной функцией ReLU
        self.conv2 = tf.keras.layers.Conv2D(channel_2, [3, 3], [1, 1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')

        # Создаем полносвязный слой с активационной функцией Softmax
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Проход данных через первый сверточный слой
        x = self.conv1(x)

        # Проход данных через второй сверточный слой
        x = self.conv2(x)

        # Выравнивание данных для подачи в полносвязный слой
        x = self.flatten(x)

        # Получение оценок классов от полносвязного слоя
        scores = self.fc(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [57]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [58]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [59]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1364245414733887, Accuracy: 15.625, Val Loss: 2.9348983764648438, Val Accuracy: 13.899999618530273
Iteration 100, Epoch 1, Loss: 2.231823205947876, Accuracy: 28.511756896972656, Val Loss: 1.9327781200408936, Val Accuracy: 38.0
Iteration 200, Epoch 1, Loss: 2.076622247695923, Accuracy: 32.16728973388672, Val Loss: 1.8325392007827759, Val Accuracy: 40.099998474121094
Iteration 300, Epoch 1, Loss: 2.0033702850341797, Accuracy: 33.88704299926758, Val Loss: 1.9023741483688354, Val Accuracy: 38.80000305175781
Iteration 400, Epoch 1, Loss: 1.9340980052947998, Accuracy: 35.76995086669922, Val Loss: 1.7348158359527588, Val Accuracy: 43.39999771118164
Iteration 500, Epoch 1, Loss: 1.8877345323562622, Accuracy: 36.882484436035156, Val Loss: 1.6605148315429688, Val Accuracy: 44.70000076293945
Iteration 600, Epoch 1, Loss: 1.8577497005462646, Accuracy: 37.75998306274414, Val Loss: 1.693747639656067, Val Accuracy: 42.5
Iteration 700, Epoch 1, Loss: 1.8324618339538574, A

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [60]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Создание модели с тремя сверточными слоями и указанием
    #количества каналов в каждом слое, а также количества классов.
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Задаем оптимизатор стохастического градиентного спуска (SGD) с указанием learning_rate,
    #параметра momentum и включением Nesterov momentum.
    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True)


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.853886604309082, Accuracy: 4.6875, Val Loss: 2.980907917022705, Val Accuracy: 7.500000476837158
Iteration 100, Epoch 1, Loss: 1.8451080322265625, Accuracy: 34.91646194458008, Val Loss: 1.61293625831604, Val Accuracy: 45.69999694824219
Iteration 200, Epoch 1, Loss: 1.6930623054504395, Accuracy: 40.446205139160156, Val Loss: 1.4588758945465088, Val Accuracy: 49.70000076293945
Iteration 300, Epoch 1, Loss: 1.6146055459976196, Accuracy: 43.189369201660156, Val Loss: 1.420101284980774, Val Accuracy: 50.80000305175781
Iteration 400, Epoch 1, Loss: 1.5494968891143799, Accuracy: 45.507328033447266, Val Loss: 1.3256241083145142, Val Accuracy: 53.10000228881836
Iteration 500, Epoch 1, Loss: 1.502746820449829, Accuracy: 47.149452209472656, Val Loss: 1.3002324104309082, Val Accuracy: 56.0
Iteration 600, Epoch 1, Loss: 1.4728596210479736, Accuracy: 48.060523986816406, Val Loss: 1.2765353918075562, Val Accuracy: 55.29999542236328
Iteration 700, Epoch 1, Loss: 1.44434154

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [61]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.242793560028076, Accuracy: 10.9375, Val Loss: 2.7546486854553223, Val Accuracy: 12.899999618530273
Iteration 100, Epoch 1, Loss: 2.2291619777679443, Accuracy: 28.217823028564453, Val Loss: 1.8675251007080078, Val Accuracy: 37.599998474121094
Iteration 200, Epoch 1, Loss: 2.071199417114258, Accuracy: 31.91075897216797, Val Loss: 1.8662388324737549, Val Accuracy: 40.5
Iteration 300, Epoch 1, Loss: 1.995688796043396, Accuracy: 34.08430099487305, Val Loss: 1.8575727939605713, Val Accuracy: 37.0
Iteration 400, Epoch 1, Loss: 1.9292712211608887, Accuracy: 35.9375, Val Loss: 1.723901391029358, Val Accuracy: 42.5
Iteration 500, Epoch 1, Loss: 1.8869705200195312, Accuracy: 36.979164123535156, Val Loss: 1.6702359914779663, Val Accuracy: 42.599998474121094
Iteration 600, Epoch 1, Loss: 1.8579471111297607, Accuracy: 37.744384765625, Val Loss: 1.6979011297225952, Val Accuracy: 42.79999923706055
Iteration 700, Epoch 1, Loss: 1.8308104276657104, Accuracy: 38.567672729492

Альтернативный менее гибкий способ обучения:

In [62]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 29ms/step - loss: 1.6410 - sparse_categorical_accuracy: 0.4374


[1.6409966945648193, 0.4374000132083893]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [63]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (28,28,1)  # Форма входных данных: изображения размером 28x28 с одним каналом (черно-белые)
    channel_1, channel_2, num_classes = 28, 14, 10  # Количество фильтров для первого и второго сверточных слоев, а также количество классов
    initializer = tf.initializers.VarianceScaling(scale=2.0)  # Инициализация весов с учетом дисперсии

    # Создаем модель, задавая порядок слоев
    layers = [
        tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Flatten(),  # Преобразование входных данных в плоский формат перед полносвязным слоем
        tf.keras.layers.Dense(num_classes,
                              activation='softmax',
                              kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers)



    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Инициализируем оптимизатор стохастического градиентного спуска (SGD) с заданной скоростью обучения (learning_rate).
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True)


    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.441401243209839, Accuracy: 10.9375, Val Loss: 2.639439105987549, Val Accuracy: 10.40000057220459
Iteration 100, Epoch 1, Loss: 1.9873087406158447, Accuracy: 29.935026168823242, Val Loss: 1.7885186672210693, Val Accuracy: 38.60000228881836
Iteration 200, Epoch 1, Loss: 1.8513487577438354, Accuracy: 34.84918975830078, Val Loss: 1.65852689743042, Val Accuracy: 42.69999694824219
Iteration 300, Epoch 1, Loss: 1.7804819345474243, Accuracy: 37.46885299682617, Val Loss: 1.612234354019165, Val Accuracy: 43.20000076293945
Iteration 400, Epoch 1, Loss: 1.7214505672454834, Accuracy: 39.42877197265625, Val Loss: 1.560039758682251, Val Accuracy: 45.79999923706055
Iteration 500, Epoch 1, Loss: 1.6798452138900757, Accuracy: 40.87138366699219, Val Loss: 1.514772891998291, Val Accuracy: 46.5
Iteration 600, Epoch 1, Loss: 1.6503098011016846, Accuracy: 41.919715881347656, Val Loss: 1.4978135824203491, Val Accuracy: 48.0
Iteration 700, Epoch 1, Loss: 1.625391960144043, Accurac

In [64]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 4s 13ms/step - loss: 1.3700 - sparse_categorical_accuracy: 0.5154


[1.3700116872787476, 0.5153999924659729]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [65]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [66]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.7862119674682617, Accuracy: 9.375, Val Loss: 2.91473388671875, Val Accuracy: 14.200000762939453
Iteration 100, Epoch 1, Loss: 2.215261936187744, Accuracy: 28.558168411254883, Val Loss: 1.9318674802780151, Val Accuracy: 38.0
Iteration 200, Epoch 1, Loss: 2.0586726665496826, Accuracy: 32.06623077392578, Val Loss: 1.8594584465026855, Val Accuracy: 38.20000076293945
Iteration 300, Epoch 1, Loss: 1.9893776178359985, Accuracy: 33.996055603027344, Val Loss: 1.8915715217590332, Val Accuracy: 37.099998474121094
Iteration 400, Epoch 1, Loss: 1.926142930984497, Accuracy: 35.80891418457031, Val Loss: 1.7498408555984497, Val Accuracy: 42.099998474121094
Iteration 500, Epoch 1, Loss: 1.8832279443740845, Accuracy: 36.96357345581055, Val Loss: 1.6680623292922974, Val Accuracy: 42.89999771118164
Iteration 600, Epoch 1, Loss: 1.8538936376571655, Accuracy: 37.80937957763672, Val Loss: 1.702332615852356, Val Accuracy: 42.5
Iteration 700, Epoch 1, Loss: 1.8272018432617188, Acc

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [83]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()

        channel_1, channel_2, num_classes = 28, 14, 10
        dp_rate = 0.2
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, [7,7], [1,1], padding='same',
                                  kernel_initializer=initializer)
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.relu1 = tf.keras.layers.ReLU()
        self.dp1 = tf.keras.layers.Dropout(rate=dp_rate)

        self.conv2 = tf.keras.layers.Conv2D(channel_2, [5,5], [1,1], padding='same',
                                  kernel_initializer=initializer)
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.relu2 = tf.keras.layers.ReLU()
        self.dp2 = tf.keras.layers.Dropout(rate=dp_rate)

        self.conv3 = tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                                  kernel_initializer=initializer)
        self.pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.relu3 = tf.keras.layers.ReLU()
        self.dp3 = tf.keras.layers.Dropout(rate=dp_rate)

        self.fl = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128,
                                  activation='relu',
                                  kernel_initializer=initializer)
        self.dp_fc1 = tf.keras.layers.Dropout(rate=dp_rate)
        self.fc2 = tf.keras.layers.Dense(num_classes,
                                  activation='softmax',
                                  kernel_initializer=initializer)

    def call(self, input_tensor, training=False):
        x = input_tensor

        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)
        x = self.dp1(x)

        x = self.conv2(x)
        x = self.pool2(x)
        x = self.relu2(x)
        x = self.dp2(x)

        x = self.conv3(x)
        x = self.pool3(x)
        x = self.relu3(x)
        x = self.dp3(x)

        x = self.fl(x)
        x = self.fc1(x)
        x = self.dp_fc1(x)
        x = self.fc2(x)

        return x
    
print_every = 700
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.643627882003784, Accuracy: 9.375, Val Loss: 4.097959041595459, Val Accuracy: 14.800000190734863
Iteration 700, Epoch 1, Loss: 1.6226614713668823, Accuracy: 45.99010467529297, Val Loss: 1.2286123037338257, Val Accuracy: 56.80000305175781
Iteration 1400, Epoch 2, Loss: 1.1541874408721924, Accuracy: 59.40206527709961, Val Loss: 1.1353862285614014, Val Accuracy: 60.5
Iteration 2100, Epoch 3, Loss: 1.0259854793548584, Accuracy: 63.886756896972656, Val Loss: 1.0874015092849731, Val Accuracy: 62.400001525878906
Iteration 2800, Epoch 4, Loss: 0.9501389265060425, Accuracy: 66.805419921875, Val Loss: 1.037643551826477, Val Accuracy: 64.0
Iteration 3500, Epoch 5, Loss: 0.9029805660247803, Accuracy: 68.76429748535156, Val Loss: 1.0659476518630981, Val Accuracy: 64.0
Iteration 4200, Epoch 6, Loss: 0.8641563653945923, Accuracy: 70.2451171875, Val Loss: 1.0375555753707886, Val Accuracy: 64.4000015258789
Iteration 4900, Epoch 7, Loss: 0.8342071771621704, Accuracy: 70.6864

Опишите все эксперименты, результаты. Сделайте выводы.

В данной работе была реализована сверточная нейронная сеть с двумя сверточными слоями и полносвязным слоем. Архитектура нейронной сети включает в себя следующие слои:

- Каждый сверточный слой использует активацию ReLU. После каждого сверточного слоя применяется пакетная нормализация и слой Dropout с коэффициентом отсечения 0.2.

- После первого сверточного слоя идет второй сверточный слой с аналогичными параметрами.

- Полносвязный слой: После сверточных слоев данные проходят через слой Flatten для преобразования в одномерный вектор, затем данные передаются в полносвязный слой с функцией активации softmax для классификации на выходе.

- Включили между Dense-слоёв Dropout

- Добавили relu с постепенно уменьшающимся размером ядра, например 7x7 -> 5x5 -> 3x3
